In [20]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import itertools as it
import requests
import re
from fake_useragent import UserAgent

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
#driver = webdriver.Chrome("C:\webdrivers", chrome_options=options)
driver = webdriver.Chrome(executable_path=r'C:/webdrivers/chromedriver.exe')


#token = "https://www.seloger.com/immobilier/locations/immo-paris-75/bien-appartement/?LISTING-LISTpg="
token ='https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg='

def get_pages(token, nb):
    pages = []
    for i in range(1,nb+1):
        j = token + str(i)
        pages.append(j)
    return pages

pages = get_pages(token,10)
print(pages)
proxies = pd.read_csv('proxy_list.txt', header = None)
proxies = proxies.values.tolist()
proxies = list(it.chain.from_iterable(proxies))

def get_info (pages,proxies) : 
    
    parameters = ['data-prix','data-idannonce','data-nb_chambres','data-nb_pieces','data-surface','data-link']
    df = pd.DataFrame(columns =parameters,index=None)
    ua = UserAgent()
    proxy_pool = it.cycle(proxies)
    
    while len(pages) > 0:
        for i in pages :
            
            # itération dans un liste de proxies    
                proxy = next(proxy_pool)

                try :
                    #response = requests.get(i,proxies={"http": proxy, "https": proxy}, headers={'User-Agent': ua.random},timeout=5)
                    #time.sleep(random.randrange(1,5))

                    #chrome_options = change_proxy (proxy)
                    #driver = webdriver.Chrome(options=chrome_options)

                    driver.get(i)
                    
                    html_page = driver.page_source
                    soup = BeautifulSoup(html_page)
                    #print(soup.prettify())
                    
                    
                    for elem_prix,elem,elem_link in it.zip_longest(\
                                             soup.find_all('div', class_=re.compile("Price__Label-sc-1g9fitq-1"))\
                                            ,soup.find_all('ul', class_=re.compile("ContentZone__Tags-wghbmy-8"))\
                                            ,soup.find_all('div', class_=re.compile("Card__ContentZone-sc-7insep-3"))):
                        
                        try :
                            l = []
                            info = soup.find_all ('li')
                            info_text=elem.text
                            if ('ch' in info_text) and ('m²' in info_text) and ('p' in info_text):
                                
                                
                                temp=re.sub('\s+', '', elem_prix.text)
                                temp=temp.replace("€","")
                                num_prix= int(temp)

                                result = re.search('ch(.*)m²', info_text)
                                num_m = result.group(1).strip()
                                #print ("nbre de m² : "+num_m+'m²')

                                result = re.search('p(.*)ch', info_text)
                                num_ch = result.group(1).strip()
                                #print ("nbre de chambres : "+num_ch)

                                split = info_text.split("p");
                                num_piece = split[0];
                                #print ("nbre de pièces : "+num_piece)
                                
                                link = elem_link.find('a',attrs={'name':'classified-link'})
                                link_val=link['href']
                                
                                df = df.append({'data-prix': num_prix, 'data-idannonce': 0, \
                                                'data-nb_chambres': num_ch, 'data-nb_pieces' : num_piece,\
                                                'data-surface' : num_m,'data-link': link_val},ignore_index=True)

                            else : 
                                
                                print('no info')
                        except : 
                            #print("data retreived fail")
                            pass
    
                    print (i)
                    pages.remove(i)
                    print("sucessufully removed")
                except :
                    print("Erreur de connexion, prochain proxy")
                    
        
        
    return df

['https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=1', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=2', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=3', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=4', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=5', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=6', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=7', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=8', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=9', 'https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=10']


In [21]:
df_final=get_info (pages,proxies)

no info
no info
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=1
sucessufully removed
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=3
sucessufully removed
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=5
sucessufully removed
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=7
sucessufully removed
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=9
sucessufully removed
no info
no info
no info
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=2
sucessufully removed
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=6
sucessufully removed
no info
https://www.seloger.com/immobilier/achat/bien-appartement/ile-de-france.htm?LISTING-LISTpg=10
sucessufully removed
no info
no info
https://www.seloger.com

In [22]:
len (df_final)

236

In [23]:
df_final.head()

,data-prix,data-idannonce,data-nb_chambres,data-nb_pieces,data-surface,data-link
0,798000,0,3,5,102,https://www.seloger.com/annonces/achat/apparte...
1,664000,0,2,3,75,https://www.seloger.com/annonces/achat/apparte...
2,577000,0,2,4,84,https://www.seloger.com/annonces/achat/apparte...
3,640000,0,1,3,53,https://www.seloger.com/annonces/achat/apparte...
4,350000,0,1,2,"22,3",https://www.seloger.com/annonces/achat/apparte...


In [24]:
import datetime

now = datetime.datetime.now()
str_now  = str(now).replace (":","-")
str_now  = str_now.replace (".","-")
print (str_now)
df_final.to_csv("achat-"+str_now+".csv",index=False,sep=';')

2020-05-21 01-37-09-992451


3101€
3101
3101
3101.0
